# SBO Markings Counts (2020)
This notebook will demonstrate how to access attribute table from shapefiles in order to create a markings asset counts for each street segment

<div style="text-align:center"><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/4/47/Road_resurfacing.jpg/1280px-Road_resurfacing.jpg'></div>

## Introduction
The purpose of this notebook is to estimate the total pavement markings asset counts over the next 5 years from ficscal year 2020 to 2024 using the markings asset feature layers from ArcGIS Online.

<i><b>Disclaimer:</b> This product is for informational purposes and may not have been prepared for or be suitable for legal, engineering, or surveying purposes. No warranty is made by the City of Austin regarding specific accuracy or completeness.</i>

## Imports
The packages used for this project are:
- [pandas](https://pandas.pydata.org/) to create dataframe of extracted table and transform the data
- [geopandas](http://geopandas.org/mapping.html) to access attribute table of 5 year selected segments for SBO
- [arcgis](https://esri.github.io/arcgis-python-api/apidoc/html/) to search for markings feature layer dataset

In [2]:
import pandas as pd
import geopandas

from arcgis.gis import GIS
from arcgis.features import FeatureLayer

%run C:\Users\Govs\Projects/FeatureLayerDataFrame.py

## Constants


In [2]:
gis = GIS("https://austin.maps.arcgis.com/home/index.html", client_id='CrnxPfTcm7Y7ZGl7')

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://austin.maps.arcgis.com/sharing/rest/oauth2/authorize?client_id=CrnxPfTcm7Y7ZGl7&response_type=code&expiration=-1&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob


Enter code obtained on signing in using SAML:  ············································································································································································································································


In [4]:
FOLDER = r'G:\ATD\Signs_and_Markings\MARKINGS\Whereabouts WORK ORDERS\SBO_2020-2024_Service_Plan'
SHP = FOLDER + r'\SBO_FiveYear_Service_Plan_09.shp'

## Import Dataset
Looking into the 2020 SBO Plans to extract Segment IDs and create list of segment IDs between bike projects and all streets

In [5]:
austin = geopandas.read_file(SHP).query('YR == 2020').filter(['SEGMENT_ID']).reset_index(drop=True)
segments = list(austin.SEGMENT_ID)
# sql no filter for existing facility on bikes variable instead of "" blank parameter
# "AND EXISTING_FACILITY LIKE 'BIKELN%'"
bikes = FeatureLayerDataFrame('bicycle_facilities').query_segments('STREET_SEGMENT_ID',segments,"")
bike_segments = list(bikes.STREET_SEGMENT_ID)

C:\Users\Govs\AppData\Roaming\Python\Python36\site-packages\arcgis\features\feature.py:612: UserWarning: The SpatialDataFrame has been deprecated. `df` property will be removed as a future release. Use `sdf` instead.
  warnings.warn(("The SpatialDataFrame has been deprecated. "
C:\Users\Govs\AppData\Roaming\Python\Python36\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")


Here it will print the number of bike lane segments affected for 2020

In [6]:
print(len(segments))
#print(len(bike_segments))
bike_df = pd.DataFrame(bike_segments)
#print(bike_segments)
#writer = pd.ExcelWriter(FOLDER +r'\bike_segments.xlsx', engine='xlsxwriter')
#bike_df.to_excel(FOLDER +r'\bike_segments.xlsx', sheet_name = 'Markings Counts')

3095


## Group and Append Datasets
For specialty markings and short line, the sum is calculated by counting the number of rows grouped by markings type. For longline, the sum is calculated by adding the total linear miles where longline exists.

In [7]:
# Method to select segments
def select_segments(seg_list,col_name):
    sp = FeatureLayerDataFrame('markings_specialty_point').query_segments('SEGMENT_ID',seg_list,'')
    sl = FeatureLayerDataFrame('markings_short_line').query_segments('SEGMENT_ID',seg_list,'')
    sp_l = FeatureLayerDataFrame('markings_specialty_line').query_segments('SEGMENT_ID',seg_list,'')
    lines = FeatureLayerDataFrame('atd_maintained_streets').query_segments('SEGMENT_ID',seg_list,'')
    sp[col_name] = sl[col_name] = sp_l[col_name] = 1
    
    # Variables
    repl = ['CROSSWALK','SCHOOL_ZONE_LINE','STOP_LINE','YIELD_LINE','Y','CURBS','DELINEATORS']
    val = ['Crosswalk','School Zone Line','Stop Line','Yield Line', 'Longline Miles','Curbs','Delineators']

    # Data Analysis
    sp_counts = sp.groupby('SPECIALTY_POINT_TYPE').count()[[col_name]].reset_index(drop=False).rename(
        columns={'SPECIALTY_POINT_TYPE':'MARKINGS TYPE'})
    sl_counts = sl.groupby('SHORT_LINE_TYPE').count()[[col_name]].reset_index(drop=False).rename(
        columns={'SHORT_LINE_TYPE':'MARKINGS TYPE'})
    sp_l_counts = sp_l.groupby('SPECIALTY_LINE_TYPE').count()[[col_name]].reset_index(drop=False).rename(
        columns={'SPECIALTY_LINE_TYPE':'MARKINGS TYPE'})
    lines_counts = lines.groupby('LONGLINE').sum()[['LINEAR_MILES']].reset_index(drop=False).query("LONGLINE == 'Y'").rename(
        columns={'LONGLINE':'MARKINGS TYPE','LINEAR_MILES':col_name})

    # Appending
    df = lines_counts.append([sp_counts,sl_counts,sp_l_counts],sort=True).replace(repl,val).set_index('MARKINGS TYPE')
    df[col_name] = df[col_name].apply(int)
    return df

This is another method to select segments with the same street name to estimate how many work orders are needed.

In [8]:
col_name = "MARKINGS"
sp = FeatureLayerDataFrame('markings_specialty_point').query_segments('SEGMENT_ID',segments,'')
sl = FeatureLayerDataFrame('markings_short_line').query_segments('SEGMENT_ID',segments,'')
sp_l = FeatureLayerDataFrame('markings_specialty_line').query_segments('SEGMENT_ID',segments,'')
lines = FeatureLayerDataFrame('atd_maintained_streets').query_segments('SEGMENT_ID',segments,'')
sp[col_name] = sl[col_name] = sp_l[col_name] = 1

# Variables
repl = ['CROSSWALK','SCHOOL_ZONE_LINE','STOP_LINE','YIELD_LINE','Y','CURBS','DELINEATORS']
val = ['Crosswalk','School Zone Line','Stop Line','Yield Line', 'Longline Miles','Curbs','Delineators']

In [75]:
# Analysis
s = 'SEGMENT_ID'
totals = sp.append([sl,sp_l],sort=True)
totals = totals.groupby(s).count()[[col_name]]
m_segs = list(totals.index)

atd = FeatureLayerDataFrame('atd_maintained_streets').query_segments('SEGMENT_ID',m_segs,'')
atd['STREET_NAME'] = atd['STREET_NAME'].str.replace(' N$','')
atd['STREET_NAME'] = atd['STREET_NAME'].str.replace(' S$','')
atd['STREET_NAME'] = atd['STREET_NAME'].str.replace(' E$','')
atd['STREET_NAME'] = atd['STREET_NAME'].str.replace(' W$','')
atd['TOTAL_SEGMENTS'] = 1
wos = atd.groupby(['STREET_NAME']).sum()[['TOTAL_SEGMENTS','LINEAR_MILES']].sort_values(by=['TOTAL_SEGMENTS'],ascending=False)
print('Total streets listed are ' + str(len(wos.index)))
atd.to_excel(FOLDER +r'\2020_Markings_street_list.xlsx', sheet_name = 'work_orders')

Total streets listed are 465


In [76]:
seg_df = FeatureLayerDataFrame('street_segment').query_segments('SEGMENT_ID',m_segs,'')
seg_df = seg_df.filter(items=['SEGMENT_ID','STREET_NAME','LEFT_BLOCK_FROM','LEFT_BLOCK_TO','STREET_TYPE'])
seg_df['STREET_NAME'] = seg_df['STREET_NAME'] +' ' + seg_df['STREET_TYPE']
min_df = seg_df.groupby(['STREET_NAME']).min(level='LEFT_BLOCK_FROM')
max_df = seg_df.groupby(['STREET_NAME']).max(level='LEFT_BLOCK_TO')

In [77]:
from_df = wos.join(min_df)
from_df = pd.merge(atd,from_df,how='right',on='SEGMENT_ID',).filter(['STREET_NAME','FROM_ST']).set_index('STREET_NAME')

to_df = wos.join(max_df)
to_df = pd.merge(atd,to_df,how='right',on='SEGMENT_ID',).filter(['STREET_NAME','TO_ST']).set_index('STREET_NAME')

total = pd.merge(to_df,from_df,right_index=True,left_index=True)
total = wos.join(total).sort_values(by=['TOTAL_SEGMENTS'],ascending=False)
total.to_excel(FOLDER +r'\2020_Markings_wo_summary.xlsx', sheet_name = 'work_orders')

In [18]:
m_counts = select_segments(m_segs,'Sum Counts')
display(m_counts)
m_counts.to_excel(FOLDER + r'\2020_Markings_Counts_wos.xlsx')

,Sum Counts
MARKINGS TYPE,
Longline Miles,73
Ahead word,1
Bicyclist symbol,321
Bike arrow,664
Blue RPM,2
Chevron crosshatch,11
Chevron symbol,239
Diagonal crosshatch,522
Diamond symbol,1


In [20]:
display(wos)

,SUM
STREET_NAME,
10TH ST W,1
PASEO SAN LUCAS LN,1
PASADENA DR,1
PARKWOOD RD,1
PARKFIELD DR,1
...,...
DUVAL RD,1
DRUMMOND DR,1
DOMINIQUE DR,1


## Display Markings Count Table
The final dataset will include sum of all, sum of bike lanes, and sum without bike lanes. This is to notify Active Transportation of any operations from SBO that would require them to make plans

Here we create two columns. One including all streets, and another that only includes bike lanes.

In [56]:
df = select_segments(segments,'Sum of All')
df_bikes = select_segments(bike_segments,'Sum of Bike Lane')

,Sum of All
MARKINGS TYPE,
Longline Miles,140
Ahead word,1
Bicyclist symbol,321
Bike arrow,664
Blue RPM,2
Chevron crosshatch,11
Chevron symbol,239
Diagonal crosshatch,522
Diamond symbol,1


In [38]:
df_final = df.merge(df_bikes,on='MARKINGS TYPE')
df_final['Sum without Bike Lanes'] = df_final["Sum of All"] - df_final["Sum of Bike Lane"]

In [39]:
display(df_final)

,Sum of All,Sum of Bike Lane,Sum without Bike Lanes
MARKINGS TYPE,,,
Longline Miles,140,64,76
Bicyclist symbol,321,236,85
Bike arrow,664,485,179
Blue RPM,2,1,1
Chevron symbol,239,127,112
Diagonal crosshatch,522,401,121
Diamond symbol,1,1,0
Green launch pad,10,2,8
Green pad,157,61,96


## Export to Excel

In [153]:
writer = pd.ExcelWriter(FOLDER +r'\2020_Markings_Counts.xlsx', engine='xlsxwriter')
df_final.to_excel(writer, sheet_name = 'Markings Counts')
wb = writer.book
ws = writer.sheets['Markings Counts']
ws.set_column('A:D',29)
writer.save()